<a href="https://colab.research.google.com/github/djswoosh/Uni-Bits/blob/main/K_NN_with_Iris_Dataset_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#InitialImports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [9]:
# Load iris data and store in data frame

from sklearn import datasets


iris = datasets.load_iris()



In [12]:
df = pd.DataFrame(data=iris.data, columns = iris.feature_names)
df['target'] = iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [13]:
#Separate X and y data

x = df.drop('target', axis=1)
y = df.target

In [15]:
# Calculate distacne between two points

def minkowski_distance(a, b, p = 1):

  #Store the number of dimensions

  dim = len(a)

  #Set initial distance to 0

  distance = 0


  #calculate minkowski distacne using parameter p

  for d in range(dim):

    distance += abs (a[d] - b [d])**p

  distance = distance**(1/p)

  return distance


In [16]:
#Test the function

minkowski_distance(a = x.iloc[0], b = x.iloc[1], p = 1)

0.6999999999999993

In [20]:
#Define an arbitraty test point

test_pt = [4.8, 2.7, 2.5, 0.7]

#Calculate distance between test_pt and all points in x

distances = []

for i in x.index:

  distances.append(minkowski_distance(test_pt, x.iloc[i]))

df_dists = pd.DataFrame(data=distances, index = x.index, columns = ['dist'])
df_dists.head()


,dist
0,2.7
1,2.0
2,2.3
3,2.1
4,2.7


In [22]:
#Find the 5 nearest neighbors

df_nn = df_dists.sort_values(by=['dist'], axis = 0) [ :5]
df_nn

,dist
98,1.4
57,1.5
93,1.7
24,1.8
30,1.8


In [24]:
from collections import Counter

#Create counter object to track the labels

counter = Counter(y[df_nn.index])

#Get most common label of all the nearest neighbors

counter.most_common()[0][0]


1

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data - 75% train, 25% test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,
                                                   random_state=1)

# Scale the x data

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [26]:
def knn_predict(x_train, x_test, y_train, y_test, k, p):
    
    # Counter to help with label voting
    from collections import Counter
    
    # Make predictions on the test data
    # Need output of 1 prediction per test data point
    y_hat_test = []

    for test_point in x_test:
        distances = []

        for train_point in x_train:
            distance = minkowski_distance(test_point, train_point, p=p)
            distances.append(distance)
        
        # Store distances in a dataframe
        df_dists = pd.DataFrame(data=distances, columns=['dist'], 
                                index=y_train.index)
        
        # Sort distances, and only consider the k closest points
        df_nn = df_dists.sort_values(by=['dist'], axis=0)[:k]

        # Create counter object to track the labels of k closest neighbors
        counter = Counter(y_train[df_nn.index])

        # Get most common label of all the nearest neighbors
        prediction = counter.most_common()[0][0]
        
        # Append prediction to output list
        y_hat_test.append(prediction)
        
    return y_hat_test


# Make predictions on test dataset
y_hat_test = knn_predict(x_train, x_test, y_train, y_test, k=5, p=1)

print(y_hat_test)

[0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0]


In [27]:
# Get test accuracy score

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_hat_test))

0.9736842105263158


In [28]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5, p=1)
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)

print(f"Sklearn KNN Accuracy: {accuracy_score(y_test, y_pred_test)}")

Sklearn KNN Accuracy: 0.9736842105263158
